# Setup

In [2]:
import sys
import os

# llama-cpp-python 로깅 숨기기
sys.stderr = open(os.devnull, "w")

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

# 모델 불러오기

In [4]:
import os

os.cpu_count()

28

In [5]:
from llama_cpp import Llama
from huggingface_hub import snapshot_download
import os

# 모델 다운로드
model_name_1 = "seungdang/gemma3-miniproj3-finetune-model2-gguf"
model_name_2 = "INe904/gemma3-finetune-recipe2-gguf"

path_1 = snapshot_download(repo_id=model_name_1, local_dir="local_model_1", revision="main")
path_2 = snapshot_download(repo_id=model_name_2, local_dir="local_model_2", revision="main")

# gguf 경로 찾기
def find_gguf_path(directory):
    for file in os.listdir(directory):
        if file.endswith(".gguf"):
            return os.path.join(directory, file)
    raise FileNotFoundError("GGUF 파일을 찾을 수 없습니다.")

model_path_1 = find_gguf_path("local_model_1")
model_path_2 = find_gguf_path("local_model_2")

# 모델 로딩
llm_1 = Llama(model_path=model_path_1, n_ctx=2048, n_threads=20)
llm_2 = Llama(model_path=model_path_2, n_ctx=2048, n_threads=20)

# 모델 로딩 확인
print("Model 1 and Model 2 are successfully loaded.")

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

(…)mma3-miniproj3-finetune-model2.Q8_0.gguf:  37%|###6      | 1.51G/4.13G [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

gemma3-finetune-recipe2.Q8_0.gguf:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/601 [00:00<?, ?B/s]

Model 1 and Model 2 are successfully loaded.


# GPT (gpt-4o-mini) 모델로 평가하는 함수

In [6]:
from openai import OpenAI

# GPT-4o-mini 모델 평가 함수
def evaluate_with_gpt4o(question, answer1, answer2):
    # OpenAI GPT-4o-mini API 호출
    client = OpenAI() 
    
    # GPT-4o-mini 모델을 사용하여 평가
    response = client.chat.completions.create(
        model="gpt-4o-mini",  # GPT-4o-mini 모델
        messages=[
            {"role": "system", "content": "당신은 한국 요리 전문가입니다. 전통적인 요리 방법, 재료, 기술에 대해 매우 잘 알고 있으며, 제공된 레시피를 평가하는 것이 임무입니다. 레시피의 명확성, 정확성, 그리고 적절한 요리 용어 사용에 대해 평가하세요. 요리 지침이 상세하고 정확하게 작성되어 있는지, 재료와 도구 사용이 적절한지, 조리 시간이 정확한지 평가해주세요. 또한, 언어가 올바르고 쉽게 이해될 수 있는지 평가하세요."},
            {"role": "user", "content": f"다음 레시피를 전문 한국 요리 전문가로서 평가해주세요:\n\n질문: {question}\n\n답변 1: {answer1}\n\n답변 2: {answer2}\n\n각 답변을 평가하여, 어떤 모델이 더 우수한 성능을 보였는지 평가해주세요."}
        ],
        max_tokens=1000,  # 최대 출력 길이
        temperature=0.7  # 적당한 창의성을 설정
    )

    # 응답에서 평가 결과 가져오기
    evaluation = response.choices[0].message.content
    return evaluation

# 불러온 모델들로 추론

In [7]:
# 질문 리스트
questions = [
    "양파, 마늘, 그리고 계란을 사용해서 20분 이내에 후라이팬만 이용해 만들 수 있는 요리를 추천해주세요.",
    "닭고기와 감자, 그리고 30분의 시간이 있을 때, 오븐을 이용해서 만들 수 있는 요리 레시피를 알려주세요.",
    "고기와 채소를 사용하여 15분 안에 전자레인지로 만들 수 있는 간단한 요리를 추천해주세요.",
    "소고기, 감자, 그리고 40분의 시간이 있을 때, 후라이팬만 사용해서 만들 수 있는 요리를 추천해주세요."
]

# 답변을 저장할 리스트
answers_model_1 = []
answers_model_2 = []

# 각 질문에 대해 두 모델이 추론한 결과를 저장
for idx, question in enumerate(questions, start=1):
    # 모델 1 추론
    response_1 = llm_1.create_chat_completion(
        messages=[{"role": "user", "content": question}],
        max_tokens=1000,
        temperature=0.7,
    )
    answer_1 = response_1['choices'][0]['message']['content']
    answers_model_1.append(answer_1)

    # 모델 2 추론
    response_2 = llm_2.create_chat_completion(
        messages=[{"role": "user", "content": question}],
        max_tokens=1000,
        temperature=0.7,
    )
    answer_2 = response_2['choices'][0]['message']['content']
    answers_model_2.append(answer_2)

    print(f"{idx}번째 결과 저장 완료!!\n")

1번째 결과 저장 완료!!

2번째 결과 저장 완료!!

3번째 결과 저장 완료!!

4번째 결과 저장 완료!!



In [8]:
# 각 질문에 대해 두 모델의 답변을 GPT-4o로 평가
for question, answer1, answer2 in zip(questions, answers_model_1, answers_model_2):
    print(f"\n[질문]\n{question}\n")
    
    print("[모델 1의 답변]")
    print(answer1, "\n")

    print("[모델 2의 답변]")
    print(answer2, "\n")

    print("[GPT-4o의 평가]")
    evaluation = evaluate_with_gpt4o(question, answer1, answer2)
    print(evaluation)
    print("=" * 120)


[질문]
양파, 마늘, 그리고 계란을 사용해서 20분 이내에 후라이팬만 이용해 만들 수 있는 요리를 추천해주세요.

[모델 1의 답변]
네, 양파, 마늘, 계란을 활용하여 후라이팬으로 20분 이내에 만들 수 있는 요리로는 다음과 같은 것들이 있습니다.

**1. 마늘 양파 계란 볶음밥 (15분)**

*   **재료:** 밥 2공기, 양파 1/2개, 마늘 2-3알, 계란 2개, 소금, 후추, 기름
*   **조리 과정:**
    1.  기름을 팬에 약한 불로 두르고 다진 마늘, 양파를 넣고 볶습니다. 양파가 투명해질 때까지 약 5분 정도, 마늘은 향이 올라오면 섞어 볶습니다.
    2.  계란을 풀어 팬에 넣고 스크램블 에그를 만듭니다.
    3.  밥을 넣고 모든 재료를 잘 섞어가며 볶습니다. 소금, 후추로 간을 맞추고, 원하는 경우 추가 재료(햄, 야채 등)를 넣어도 좋습니다.
*   **팁:** 밥은 미리 해 놓으면 요리를 더욱 빠르게 진행할 수 있습니다.

**2. 계란 토우 (마늘 녹두무침) (20분)**

*   **재료:** 계란 3개, 양파 1/4개, 마늘 1-2알, 고추가루 1-2스푼, 참기름, 깨소금, 소금
*   **조리 과정:**
    1.  계란을 삶아 식힌 후 으깨줍니다.
    2.  양파와 마늘은 잘 다져줍니다.
    3.  으깬 계란, 다진 양파, 다진 마늘, 고추가루, 참기름, 깨소금을 섞어 양념장을 만듭니다. 소금으로 간을 살짝 맞춥니다.
*   **팁:** 남은 계란과 잘 보관할 수 있으며, 밥 반찬으로도 좋고 김치와 함께 먹으면 밥 한공기 뚝딱!

**3. 마늘 양파 계란 볶음 (15분)**

*   **재료:** 돼지고기 다짐육 150g, 양파 1/2개, 마늘 2-3알, 간장 2스푼, 굴 소스 1스푼, 후레이크(선택), 배즙(선택), 소금, 식용유, 정란 노 액 (선택)
*   **조리 과정:**
    1.  양파와 마늘은 잘게 썰어 준비합니다.
    2.  고추기름을 팬에 약한 불에서 고소한 향이 나도록 